In [28]:
#@title login google 帳號, 按左邊按鈕執行。之後，輸入組合, GO，輸入組合, GO, ...
from itertools import combinations, product, chain, permutations
from collections import Counter
from random import sample
import pickle
from pathlib import Path

hand_names=["無", "對", "兩對", "三條", "順", "同花", "葫蘆", "鐵支", "同花順"]
hand_names_eng=["High Card", "One pair", "Two pair", "Three of a kind", "Straight", "Flush", "Full house", "Four of a kind", "Straight flush"]

suit_names=list("♠♥♦♣")
suit_names_eng = list("SHDC")
playing_card_unicodes={'S':'🂡🂢🂣🂤🂥🂦🂧🂨🂩🂪🂫🂬🂭🂮',
                       'H':'🂱🂲🂳🂴🂵🂶🂷🂸🂹🂺🂻🂼🂽🂾',
                       'D':'🃁🃂🃃🃄🃅🃆🃇🃈🃉🃊🃋🃌🃍🃎',
                       'C':'🃑🃒🃓🃔🃕🃖🃗🃘🃙🃚🃛🃜🃝🃞'
}

def eng_to_unicode(s):
  s = s.replace("?", "🃏")
  #return playing_card_unicodes[s[0]][rank_name_to_idx[s[1]]]
  for a,b in zip(suit_names_eng, suit_names):
    s = s.replace(a, b)
  return s
rank_names=list("89TJQKA")

HAND ={n:i for i, n in enumerate(hand_names)}
NUM_SUITS = len(suit_names)
suit_list = list(range(NUM_SUITS))
NUM_RANKS = len(rank_names)
rank_list = list(range(NUM_RANKS))
rank_name_to_idx={n:i for i, n in enumerate(rank_names)}

all_cards = list(product(suit_list, rank_list))
all_cards_2joker = all_cards+[(-1,-1)]*2
assert len(all_cards_2joker)==30
all_hand_combinations = list(chain(*[combinations(all_cards, i) for i in range(6)]))
# cards are a pair of numbers
# jokers are (-1,-1), (-1,-2)

def compute_hand(cards):
    ranks = [i for s,i in cards]
    NUM_DIFF_RANKS = len(set(ranks))
    NUM_CARDS = len(cards)    
    if NUM_DIFF_RANKS == NUM_CARDS: # no duplicate ranks
        sorted_ranks = tuple(sorted(ranks, reverse=True))
        if NUM_CARDS < 5:
            return (HAND['無'], sorted_ranks)        
        is_flush =  all(s==cards[0][0] for s,i in cards)            
        is_straight = sorted_ranks[0]==sorted_ranks[4]+4
        if is_straight:
            hand_name = "同花順" if is_flush else "順"
        else:
            hand_name = "同花" if is_flush else "無"            
        return (HAND[hand_name], sorted_ranks)
    # with duplicate ranks
    hist = Counter(ranks).most_common()
    kickers = tuple(sorted([i for i, n in hist if n==1], reverse=True))
    if NUM_DIFF_RANKS == len(kickers)+1: # 對、三條、四條
        hand_name = ['對', '三條', '鐵支'][hist[0][1]-2]        
        return (HAND[hand_name], (hist[0][0],)+kickers)
    assert NUM_DIFF_RANKS == len(kickers)+2
    if hist[0][1]==3: # 葫蘆
        assert hist[1][1]==2
        return (HAND['葫蘆'], (hist[0][0], hist[1][0])+kickers)
    # 兩對
    r1, r2 = hist[0][0], hist[1][0]
    if r1 < r2:
        r1, r2 = r2, r1
    return (HAND['兩對'], (r1, r2)+kickers)
computed_hands={}
for c in all_hand_combinations:
    computed_hands[c] = compute_hand(c)
    
def test_hands(computed_hands):
    truth = pickle.loads(Path('computed_hands_original.pkl').read_bytes())
    for k in truth:
        if truth[k] != computed_hands[k]:
            print(f'Test error: {k=} {truth[k]=} {computed_hands[k]=}')
            
#test_hands(computed_hands)

In [29]:
# add jokers
# add one_joker
for base_cards in chain(*[combinations(all_cards, i) for i in range(5)]):
    joker_cards = ((-1,-1),)+base_cards
    other_cards = set(all_cards)-set(base_cards)
    best_hand = computed_hands[base_cards]
    best_cards = base_cards
    #print(f'{base_cards=}')
    for c in other_cards:        
        cards = tuple(sorted(base_cards+(c,)))
        hand = computed_hands[cards]
        if hand > best_hand:
            best_hand = hand
            best_cards = cards
    #print(f'{best_cards=}, {best_hand=}')
    computed_hands[joker_cards]=best_hand
    

In [30]:
# add two_jokers
for base_cards in chain(*[combinations(all_cards, i) for i in range(4)]):
    joker_cards = ((-1,-1),(-1,-1))+base_cards
    other_cards = set(all_cards)-set(base_cards)
    best_hand = computed_hands[base_cards]
    best_cards = base_cards
    #print(f'{base_cards=}')
    for cs in combinations(other_cards, 2):
        cards = tuple(sorted(base_cards+cs))
        hand = computed_hands[cards]
        if hand > best_hand:
            best_hand = hand
            best_cards = cards
    #print(f'{best_cards=}, {best_hand=}')
    computed_hands[joker_cards]=best_hand
    

In [31]:
def card_repr(card):
    if card == (-1,-1):
        return "🃏"
    return "" if card is None else suit_names[card[0]]+rank_names[card[1]]
def cards_repr(cards):
    s = " ".join(map(card_repr, cards))
    return s
def hand_repr(hand):
    return hand_names[hand[0]]+" "+"".join(rank_names[r] for r in hand[1])


def str2cards(s, fill_none=0):
  s = s.rstrip().lstrip()
  while '  ' in s:
    s = s.replace('  ', ' ')
  c = [None if x=='N' else (-1,-1) if x=="🃏" else (suit_names.index(x[0]), rank_names.index(x[1])) for x in s.split(' ')]
  if fill_none and len(c)<fill_none:
    c = c +[None]*(fill_none-len(c))
  return c

def str2setting(s):
  s=s.replace('[','').replace(']','\n')
  setting = [str2cards(x, fill_none=6) for x in  s.splitlines() if len(x)>1]
  assert len(setting)==5
  assert all(len(x)==6 for x in setting)
  return setting

def setting_repr(setting):
  return "\n".join(cards_repr([x for x in s if x is not None]) for s in setting)

def random_setting():
  setting = sample(all_cards_2joker, 30)
  setting = [setting[i:i+6] for i in range(0, 30,6)]
  return setting

def rec(result, setting, prob=1., cards=[]):
    if len(setting)==0:
        cards = tuple(sorted(cards))
        hand = computed_hands[cards]
        result[hand] = result.get(hand, 0) + prob
        return result
    #print(setting)
    sides = setting[0]
    assert len(sides)==6
    good_sides = [s for s in sides if s is not None]
    none_prob = (6-len(good_sides))/6
    other_prob = prob/6*(1+none_prob+none_prob**2)
    rec(result, setting[1:], prob * none_prob**3, cards)
    for s in good_sides:
        rec(result, setting[1:], other_prob, cards+[s])
    return result

def compute_dist(setting):
    result = rec({}, setting)
    result = sorted(result.items())
    a = 0
    final_result = []
    for h, p in result:
        final_result.append((h, p, a))
        a += p
    return final_result
def compare_dist(dist1, dist2):
  iter_dist2 = iter(dist2)
  w1, e = 0,0
  h2 = tuple()
  #h2 = -1
  for h1, p1, a1 in dist1:
      try:
          while h2 < h1:
              h2, p2, a2 = next(iter_dist2)                
          #print('x', h1, h2, p1, p2, e)
          if h2 == h1:
              e += p1*p2
              #print(h1, h2, p1, p2, e)
          w1 += p1*a2
      except StopIteration:
          w1 += p1
  w2 = 1- e - w1
  return w1, e, w2
def compare_setting(setting1, setting2):
  dist1 = compute_dist(setting1)
  dist2 = compute_dist(setting2)
  return compare_dist(dist1, dist2)
def setting_summary(setting, eng=False):
  dist = compute_dist(setting)
  for name, name_eng in zip(hand_names, hand_names_eng):
    prob = sum(p for h,p,a in dist if h[0]==HAND[name])
    show_name = name_eng if eng else name
    print(f"{show_name:>15s}: {prob*100: >5.2f}%")
  print(setting_repr(setting))

import ipywidgets as iw
from IPython.display import display, clear_output
setting0_str = eng_to_unicode("""SA HA CA DA S9 S8
SK HK CK DK H9 H8
SQ HQ CQ DQ D9 D8
SJ HJ CJ DJ C9 C8
ST HT CT DT ? ?
""")
setting1_str = eng_to_unicode("""SA HT CJ DQ S9 S8
SK HA CT DJ H9 H8
SQ HK CA DT C9 C8
SJ HQ CK DA D9 D8
ST HJ CQ DK ? ?
""")
# 第二種組合
setting2_str = eng_to_unicode("""SA HA CA DA S9 H9
SK HK CK DK ? ?
SQ HQ CQ DQ S8 D8
SJ HJ CJ DJ H8 C8
ST HT CT DT D9 C9
""")

In [32]:
setting2_str

'♠A ♥A ♣A ♦A ♠9 ♥9\n♠K ♥K ♣K ♦K 🃏 🃏\n♠Q ♥Q ♣Q ♦Q ♠8 ♦8\n♠J ♥J ♣J ♦J ♥8 ♣8\n♠T ♥T ♣T ♦T ♦9 ♣9\n'

In [33]:
setting0 = str2setting(setting0_str)

def cards_to_buttons(cards):
  layout = iw.Layout(width='4em', height='2em', font_color='red') #set width and height
  return [iw.ToggleButton(description=card_repr(s), layout=layout) for s in cards]

def button_to_cards_str(buttons):
  return " ".join(b.description for b in buttons)

class DiceSettingUI:
  def button_clicked(self, b):
    if b['new']==False:
      self.selected -= 1
      assert self.selected>=0
      return
    self.selected += 1
    if self.selected == 1:
      return
    selected_buttons = [button for button in chain(*self.buttons) if button.value]
    if len(selected_buttons)==2:
        x, y = selected_buttons
        x.description, y.description = y.description, x.description
    if len(selected_buttons)>=2:
      for button in selected_buttons:
        button.value=False
  def get_setting(self):
    setting_str = "\n".join(button_to_cards_str(buttons) for buttons in self.buttons)
    return str2setting(setting_str)
  def __init__(self, setting, description):
    self.buttons = [cards_to_buttons(dice) for dice in setting]
    self.selected = 0
    for button in chain(*self.buttons):
      button.observe(self.button_clicked, 'value')
      button.style.text_color="green"

    self.widget = iw.VBox([iw.Label(description)]+list(map(iw.HBox, self.buttons)), layout=iw.Layout(margin="2em"))
setting1_ui = DiceSettingUI(setting0, 'Setting1')
setting2_ui = DiceSettingUI(setting0, 'Setting2')


import sys
def on_button_clicked(a):
  setting1 = setting1_ui.get_setting()
  setting2 = setting2_ui.get_setting()
  w1, e, w2 = compare_setting(setting1, setting2)

  output0.value =(f"Setting1 Win:{w1*100:.3f}%  Tie:{e*100:.3f}%     Setting2 Win:{w2*100:.3f}%")
  with output1:
    clear_output()
    print(' '*40)
    setting_summary(setting1, eng=True)
  with output2:
    clear_output()
    print(' '*40)
    setting_summary(setting2, eng=True)
output0 = iw.Label(cols=60)
output1 = iw.Output(rows=10, cols=40)
output2 = iw.Output(rows=10, cols=40)

button = iw.Button(description='GO')
button.on_click(on_button_clicked)
display(
    iw.VBox([iw.HBox([setting1_ui.widget, setting2_ui.widget]),
             button,
             output0,
             iw.HBox([output1, output2])
             ])
)